<center>
    <img src="https://gitlab.com/ibm/skills-network/courses/placeholder101/-/raw/master/labs/module%201/images/IDSNlogo.png"  width="300" alt="cognitiveclass.ai logo"  />
</center>

<h1 align=center><font size = 5>Assignment: SQL Notebook for Peer Assignment</font></h1>

Estimated time needed: **60** minutes.

## Introduction

Using this Python notebook you will:

1.  Understand the Spacex DataSet
2.  Load the dataset  into the corresponding table in a Db2 database
3.  Execute SQL queries to answer assignment questions


## Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones.

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage.

Therefore if we can determine if the first stage will land, we can determine the cost of a launch.

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.


### Download the datasets

This assignment requires you to load the spacex dataset.

In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. Click on the link below to download and save the dataset (.CSV file):

<a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01" target="_blank">Spacex DataSet</a>


### Store the dataset in database table

**it is highly recommended to manually load the table using the database console LOAD tool in DB2**.

<img src = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/images/spacexload.png">

Now open the Db2 console, open the LOAD tool, Select / Drag the .CSV file for the  dataset, Next create a New Table, and then follow the steps on-screen instructions to load the data. Name the new table as follows:

**SPACEXDATASET**

**Follow these steps while using old DB2 UI which is having Open Console Screen**

**Note:While loading Spacex dataset, ensure that detect datatypes is disabled. Later click on the pencil icon(edit option).**

1.  Change the Date Format by manually typing DD-MM-YYYY and timestamp format as DD-MM-YYYY HH\:MM:SS.

    Here you should place the cursor at Date field and manually type as DD-MM-YYYY.

2.  Change the PAYLOAD_MASS\_\_KG\_  datatype  to INTEGER.

<img src = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/images/spacexload2.png">


**Changes to be considered when having DB2 instance with the new UI having Go to UI screen**

*   Refer to this insruction in this <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Sign%20up%20for%20IBM%20Cloud%20-%20Create%20Db2%20service%20instance%20-%20Get%20started%20with%20the%20Db2%20console/instructional-labs.md.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01">link</a> for viewing  the new  Go to UI screen.

*   Later click on **Data link(below SQL)**  in the Go to UI screen  and click on **Load Data** tab.

*   Later browse for the downloaded spacex file.

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/images/browsefile.png" width="800"/>

*   Once done select the schema andload the file.

 <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/images/spacexload3.png" width="800"/>


In [1]:
!pip install sqlalchemy==1.3.9
!pip install ibm_db_sa
!pip install ipython-sql

     |████████████████████████████████| 6.0 MB 1.4 MB/s 
  Created wheel for sqlalchemy: filename=SQLAlchemy-1.3.9-cp37-cp37m-linux_x86_64.whl size=1203574 sha256=d7fa4cb8187c0e9e76025a52a62fe526adffd7ea6dc8ad50a0cdaba9ba40a21d
  Stored in directory: /root/.cache/pip/wheels/03/71/13/010faf12246f72dc76b4150e6e599d13a85b4435e06fb9e51f
Successfully built sqlalchemy
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.4.31
    Uninstalling SQLAlchemy-1.4.31:
      Successfully uninstalled SQLAlchemy-1.4.31
     |████████████████████████████████| 1.1 MB 8.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for ibm-db-sa: filename=ibm_db_sa-0.3.7-py3-none-any.whl size=29316 sha256=498650cb818184eba414a3c1dc041c0f516a105ffdcf05c339a2e08f8688d619
  Stored in directory: /root/.cache/pip/wheels/4a/e9/e7/0ee334a6cb2f09ec45978e0583

### Connect to the database

Let us first load the SQL extension and establish a connection with the database


As  IBM's DB2 guidelines on this notebook doesnt work  I will implement the SQL queries using findspark

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [4]:
!pip install -q findspark

import findspark
findspark.init()

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [7]:
!pip install -U pandasql

  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26784 sha256=da727a09f7d77ee283e7bc5b11eaffea818c4ec912fce8c0557e9eb9145e19ff
  Stored in directory: /root/.cache/pip/wheels/5c/4b/ec/41f4e116c8053c3654e2c2a47c62b4fca34cc67ef7b55deb7f
Successfully built pandasql


In [8]:
from pandasql import sqldf 
mysql = lambda q: sqldf(q, globals())

## Tasks

Now write and execute SQL queries to solve the assignment tasks.

### Task 1

##### Display the names of the unique launch sites  in the space mission


In [9]:
df = spark.read.csv("/content/Spacex.csv",header=True, inferSchema=True)

In [10]:
df.show(5)

+----------+----------+---------------+-----------+--------------------+-----------------+---------+---------------+---------------+-------------------+
|      Date|Time (UTC)|Booster_Version|Launch_Site|             Payload|PAYLOAD_MASS__KG_|    Orbit|       Customer|Mission_Outcome|   Landing _Outcome|
+----------+----------+---------------+-----------+--------------------+-----------------+---------+---------------+---------------+-------------------+
|04-06-2010|  18:45:00| F9 v1.0  B0003|CCAFS LC-40|Dragon Spacecraft...|                0|      LEO|         SpaceX|        Success|Failure (parachute)|
|08-12-2010|  15:43:00| F9 v1.0  B0004|CCAFS LC-40|Dragon demo fligh...|                0|LEO (ISS)|NASA (COTS) NRO|        Success|Failure (parachute)|
|22-05-2012|  07:44:00| F9 v1.0  B0005|CCAFS LC-40|Dragon demo fligh...|              525|LEO (ISS)|    NASA (COTS)|        Success|         No attempt|
|08-10-2012|  00:35:00| F9 v1.0  B0006|CCAFS LC-40|        SpaceX CRS-1|          

In [11]:
df.select("Launch_Site").distinct().show()

+------------+
| Launch_Site|
+------------+
|CCAFS SLC-40|
| VAFB SLC-4E|
|  KSC LC-39A|
| CCAFS LC-40|
+------------+



### Task 2

##### Display 5 records where launch sites begin with the string 'CCA'


In [12]:
df2 = df.select("*", df.Launch_Site.like("CCA%")).show(5)

+----------+----------+---------------+-----------+--------------------+-----------------+---------+---------------+---------------+-------------------+---------------------+
|      Date|Time (UTC)|Booster_Version|Launch_Site|             Payload|PAYLOAD_MASS__KG_|    Orbit|       Customer|Mission_Outcome|   Landing _Outcome|Launch_Site LIKE CCA%|
+----------+----------+---------------+-----------+--------------------+-----------------+---------+---------------+---------------+-------------------+---------------------+
|04-06-2010|  18:45:00| F9 v1.0  B0003|CCAFS LC-40|Dragon Spacecraft...|                0|      LEO|         SpaceX|        Success|Failure (parachute)|                 true|
|08-12-2010|  15:43:00| F9 v1.0  B0004|CCAFS LC-40|Dragon demo fligh...|                0|LEO (ISS)|NASA (COTS) NRO|        Success|Failure (parachute)|                 true|
|22-05-2012|  07:44:00| F9 v1.0  B0005|CCAFS LC-40|Dragon demo fligh...|              525|LEO (ISS)|    NASA (COTS)|        S

### Task 3

##### Display the total payload mass carried by boosters launched by NASA (CRS)


In [ ]:
df3 = df.filter(df["Customer"] == 'NASA (CRS)').show()

+----------+----------+---------------+------------+--------------------+-----------------+---------+----------+-------------------+--------------------+
|      Date|Time (UTC)|Booster_Version| Launch_Site|             Payload|PAYLOAD_MASS__KG_|    Orbit|  Customer|    Mission_Outcome|    Landing _Outcome|
+----------+----------+---------------+------------+--------------------+-----------------+---------+----------+-------------------+--------------------+
|08-10-2012|  00:35:00| F9 v1.0  B0006| CCAFS LC-40|        SpaceX CRS-1|              500|LEO (ISS)|NASA (CRS)|            Success|          No attempt|
|01-03-2013|  15:10:00| F9 v1.0  B0007| CCAFS LC-40|        SpaceX CRS-2|              677|LEO (ISS)|NASA (CRS)|            Success|          No attempt|
|18-04-2014|  19:25:00|        F9 v1.1| CCAFS LC-40|        SpaceX CRS-3|             2296|LEO (ISS)|NASA (CRS)|            Success|  Controlled (ocean)|
|21-09-2014|  05:52:00|  F9 v1.1 B1010| CCAFS LC-40|        SpaceX CRS-4|   

### Task 4

##### Display average payload mass carried by booster version F9 v1.1


In [ ]:
df4 = df.filter(df.Booster_Version.like("F9 v1.1%")).show()

+----------+----------+---------------+-----------+--------------------+-----------------+---------+--------------------+-------------------+--------------------+
|      Date|Time (UTC)|Booster_Version|Launch_Site|             Payload|PAYLOAD_MASS__KG_|    Orbit|            Customer|    Mission_Outcome|    Landing _Outcome|
+----------+----------+---------------+-----------+--------------------+-----------------+---------+--------------------+-------------------+--------------------+
|29-09-2013|  16:00:00| F9 v1.1  B1003|VAFB SLC-4E|            CASSIOPE|              500|Polar LEO|                 MDA|            Success|Uncontrolled (ocean)|
|03-12-2013|  22:41:00|        F9 v1.1|CCAFS LC-40|               SES-8|             3170|      GTO|                 SES|            Success|          No attempt|
|06-01-2014|  22:06:00|        F9 v1.1|CCAFS LC-40|           Thaicom 6|             3325|      GTO|             Thaicom|            Success|          No attempt|
|18-04-2014|  19:25:00

### Task 5

##### List the date when the first successful landing outcome in ground pad was acheived.

*Hint:Use min function*


In [16]:
df5 = df.filter(df["Landing _Outcome"].like("Success (ground pad)")).show(1)

+----------+----------+---------------+-----------+--------------------+-----------------+-----+--------+---------------+--------------------+
|      Date|Time (UTC)|Booster_Version|Launch_Site|             Payload|PAYLOAD_MASS__KG_|Orbit|Customer|Mission_Outcome|    Landing _Outcome|
+----------+----------+---------------+-----------+--------------------+-----------------+-----+--------+---------------+--------------------+
|22-12-2015|  01:29:00|    F9 FT B1019|CCAFS LC-40|OG2 Mission 2  11...|             2034|  LEO| Orbcomm|        Success|Success (ground pad)|
+----------+----------+---------------+-----------+--------------------+-----------------+-----+--------+---------------+--------------------+
only showing top 1 row



### Task 6

##### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [ ]:
df6 = df.filter(df["Landing _Outcome"].like("Success (drone ship)")).show()

+----------+----------+---------------+------------+--------------------+-----------------+---------+--------------------+---------------+--------------------+
|      Date|Time (UTC)|Booster_Version| Launch_Site|             Payload|PAYLOAD_MASS__KG_|    Orbit|            Customer|Mission_Outcome|    Landing _Outcome|
+----------+----------+---------------+------------+--------------------+-----------------+---------+--------------------+---------------+--------------------+
|08-04-2016|  20:43:00|  F9 FT B1021.1| CCAFS LC-40|        SpaceX CRS-8|             3136|LEO (ISS)|          NASA (CRS)|        Success|Success (drone ship)|
|06-05-2016|  05:21:00|    F9 FT B1022| CCAFS LC-40|            JCSAT-14|             4696|      GTO|SKY Perfect JSAT ...|        Success|Success (drone ship)|
|27-05-2016|  21:39:00|  F9 FT B1023.1| CCAFS LC-40|           Thaicom 8|             3100|      GTO|             Thaicom|        Success|Success (drone ship)|
|14-08-2016|  05:26:00|    F9 FT B1026| 

### Task 7

##### List the total number of successful and failure mission outcomes


In [ ]:
df7_1 = df.filter(df.Mission_Outcome.like("Success%")).show(100)
#df.Mission_Outcome.like("Failure%")

+----------+----------+---------------+------------+--------------------+-----------------+-----------+--------------------+--------------------+--------------------+
|      Date|Time (UTC)|Booster_Version| Launch_Site|             Payload|PAYLOAD_MASS__KG_|      Orbit|            Customer|     Mission_Outcome|    Landing _Outcome|
+----------+----------+---------------+------------+--------------------+-----------------+-----------+--------------------+--------------------+--------------------+
|04-06-2010|  18:45:00| F9 v1.0  B0003| CCAFS LC-40|Dragon Spacecraft...|                0|        LEO|              SpaceX|             Success| Failure (parachute)|
|08-12-2010|  15:43:00| F9 v1.0  B0004| CCAFS LC-40|Dragon demo fligh...|                0|  LEO (ISS)|     NASA (COTS) NRO|             Success| Failure (parachute)|
|22-05-2012|  07:44:00| F9 v1.0  B0005| CCAFS LC-40|Dragon demo fligh...|              525|  LEO (ISS)|         NASA (COTS)|             Success|          No attempt

In [ ]:
df7_2 = df.filter(df.Mission_Outcome.like("Failure%")).show(100)

+----------+----------+---------------+-----------+------------+-----------------+---------+----------+-------------------+--------------------+
|      Date|Time (UTC)|Booster_Version|Launch_Site|     Payload|PAYLOAD_MASS__KG_|    Orbit|  Customer|    Mission_Outcome|    Landing _Outcome|
+----------+----------+---------------+-----------+------------+-----------------+---------+----------+-------------------+--------------------+
|28-06-2015|  14:21:00|  F9 v1.1 B1018|CCAFS LC-40|SpaceX CRS-7|             1952|LEO (ISS)|NASA (CRS)|Failure (in flight)|Precluded (drone ...|
+----------+----------+---------------+-----------+------------+-----------------+---------+----------+-------------------+--------------------+



In [ ]:
import pandas as pd
import numpy as np

### Task 8

##### List the   names of the booster_versions which have carried the maximum payload mass. Use a subquery


cursor.execute("SELECT DISTINCT Booster_Version, MAX(PAYLOAD_MASS_KG_) AS [Maximum Payload Mass] FROM tblSpaceX GROUP BY Booster_Version ORDER BY [Maximum Payload Mass] DESC")
columns = [column[0] for column in cursor.description]

In [ ]:
dfx = df.select("PAYLOAD_MASS__KG_").distinct().show(200)

+-----------------+
|PAYLOAD_MASS__KG_|
+-----------------+
|             5300|
|             3000|
|             4000|
|             6500|
|             1977|
|            12530|
|              362|
|             6460|
|            12055|
|             4230|
|             2697|
|            15440|
|             3669|
|             4200|
|             6956|
|             3325|
|             3500|
|             5500|
|             2647|
|             4428|
|             2296|
|             4311|
|             5000|
|            12500|
|             1952|
|              500|
|              570|
|             4159|
|             2268|
|            15410|
|              553|
|            14932|
|             1192|
|             2150|
|            12050|
|             3310|
|             2257|
|             5800|
|             2972|
|             2205|
|             2490|
|              677|
|             5271|
|             5600|
|             2495|
|             3170|
|             5200|


In [ ]:
df8 = df.filter(df.PAYLOAD_MASS__KG_.like("15600%")).show(500)

+----------+----------+---------------+------------+--------------------+-----------------+-----+-------------------+---------------+----------------+
|      Date|Time (UTC)|Booster_Version| Launch_Site|             Payload|PAYLOAD_MASS__KG_|Orbit|           Customer|Mission_Outcome|Landing _Outcome|
+----------+----------+---------------+------------+--------------------+-----------------+-----+-------------------+---------------+----------------+
|11-11-2019|  14:56:00|  F9 B5 B1048.4|CCAFS SLC-40|Starlink 1 v1.0, ...|            15600|  LEO|             SpaceX|        Success|         Success|
|07-01-2020|  02:33:00|  F9 B5 B1049.4|CCAFS SLC-40|Starlink 2 v1.0, ...|            15600|  LEO|             SpaceX|        Success|         Success|
|29-01-2020|  14:07:00|  F9 B5 B1051.3|CCAFS SLC-40|Starlink 3 v1.0, ...|            15600|  LEO|             SpaceX|        Success|         Success|
|17-02-2020|  15:05:00|  F9 B5 B1056.4|CCAFS SLC-40|Starlink 4 v1.0, ...|            15600|  L

### Task 9

##### List the failed landing_outcomes in drone ship, their booster versions, and launch site names for in year 2015


In [28]:
 df9 = df.filter((df["Landing _Outcome"].like("Failure (drone ship)")) & df["Date"].like("%2015")).show()

+----------+----------+---------------+-----------+------------+-----------------+---------+----------+---------------+--------------------+
|      Date|Time (UTC)|Booster_Version|Launch_Site|     Payload|PAYLOAD_MASS__KG_|    Orbit|  Customer|Mission_Outcome|    Landing _Outcome|
+----------+----------+---------------+-----------+------------+-----------------+---------+----------+---------------+--------------------+
|10-01-2015|  09:47:00|  F9 v1.1 B1012|CCAFS LC-40|SpaceX CRS-5|             2395|LEO (ISS)|NASA (CRS)|        Success|Failure (drone ship)|
|14-04-2015|  20:10:00|  F9 v1.1 B1015|CCAFS LC-40|SpaceX CRS-6|             1898|LEO (ISS)|NASA (CRS)|        Success|Failure (drone ship)|
+----------+----------+---------------+-----------+------------+-----------------+---------+----------+---------------+--------------------+



In [36]:
 df9 = df.filter(df["Date"].like("%2015")).show()

+----------+----------+---------------+-----------+--------------------+-----------------+---------+--------------------+-------------------+--------------------+
|      Date|Time (UTC)|Booster_Version|Launch_Site|             Payload|PAYLOAD_MASS__KG_|    Orbit|            Customer|    Mission_Outcome|    Landing _Outcome|
+----------+----------+---------------+-----------+--------------------+-----------------+---------+--------------------+-------------------+--------------------+
|10-01-2015|  09:47:00|  F9 v1.1 B1012|CCAFS LC-40|        SpaceX CRS-5|             2395|LEO (ISS)|          NASA (CRS)|            Success|Failure (drone ship)|
|11-02-2015|  23:03:00|  F9 v1.1 B1013|CCAFS LC-40|              DSCOVR|              570|      HEO|U.S. Air Force NA...|            Success|  Controlled (ocean)|
|02-03-2015|  03:50:00|  F9 v1.1 B1014|CCAFS LC-40|ABS-3A Eutelsat 1...|             4159|      GTO|        ABS Eutelsat|            Success|          No attempt|
|14-04-2015|  20:10:00

### Task 10

##### Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order


SELECT COUNT(Landing_Outcome) AS sl FROM dbo.tblSpaceX WHERE (Landing_Outcome LIKE '%Success%') AND (Date >'04-06-2010') AND (Date < '20-03-2017')

In [52]:
 df10 = df.filter((df["Landing _Outcome"].like("Success (ground pad)")) & (df["Date"]<"04-01-2010")).show()

+----------+----------+---------------+-----------+-------------+-----------------+---------+----------+---------------+--------------------+
|      Date|Time (UTC)|Booster_Version|Launch_Site|      Payload|PAYLOAD_MASS__KG_|    Orbit|  Customer|Mission_Outcome|    Landing _Outcome|
+----------+----------+---------------+-----------+-------------+-----------------+---------+----------+---------------+--------------------+
|01-05-2017|  11:15:00|  F9 FT B1032.1| KSC LC-39A|      NROL-76|             5300|      LEO|       NRO|        Success|Success (ground pad)|
|03-06-2017|  21:07:00|  F9 FT B1035.1| KSC LC-39A|SpaceX CRS-11|             2708|LEO (ISS)|NASA (CRS)|        Success|Success (ground pad)|
+----------+----------+---------------+-----------+-------------+-----------------+---------+----------+---------------+--------------------+



In [54]:
 df10 = df.filter((df["Landing _Outcome"].like("Success (ground pad)")) & (df["Date"]>'04-06-2010') & (df["Date"]<'20-03-2017')).show()

+----------+----------+---------------+------------+------------------+-----------------+---------+----------------+--------------------+--------------------+
|      Date|Time (UTC)|Booster_Version| Launch_Site|           Payload|PAYLOAD_MASS__KG_|    Orbit|        Customer|     Mission_Outcome|    Landing _Outcome|
+----------+----------+---------------+------------+------------------+-----------------+---------+----------------+--------------------+--------------------+
|18-07-2016|  04:45:00|  F9 FT B1025.1| CCAFS LC-40|      SpaceX CRS-9|             2257|LEO (ISS)|      NASA (CRS)|             Success|Success (ground pad)|
|19-02-2017|  14:39:00|  F9 FT B1031.1|  KSC LC-39A|     SpaceX CRS-10|             2490|LEO (ISS)|      NASA (CRS)|             Success|Success (ground pad)|
|14-08-2017|  16:31:00|  F9 B4 B1039.1|  KSC LC-39A|     SpaceX CRS-12|             3310|LEO (ISS)|      NASA (CRS)|             Success|Success (ground pad)|
|07-09-2017|  14:00:00|  F9 B4 B1040.1|  KSC L

In [53]:
 df10 = df.filter((df["Landing _Outcome"].like("Success (ground pad)")) & df["Date"].like("%201%")).show(3)

+----------+----------+---------------+-----------+--------------------+-----------------+---------+----------+---------------+--------------------+
|      Date|Time (UTC)|Booster_Version|Launch_Site|             Payload|PAYLOAD_MASS__KG_|    Orbit|  Customer|Mission_Outcome|    Landing _Outcome|
+----------+----------+---------------+-----------+--------------------+-----------------+---------+----------+---------------+--------------------+
|22-12-2015|  01:29:00|    F9 FT B1019|CCAFS LC-40|OG2 Mission 2  11...|             2034|      LEO|   Orbcomm|        Success|Success (ground pad)|
|18-07-2016|  04:45:00|  F9 FT B1025.1|CCAFS LC-40|        SpaceX CRS-9|             2257|LEO (ISS)|NASA (CRS)|        Success|Success (ground pad)|
|19-02-2017|  14:39:00|  F9 FT B1031.1| KSC LC-39A|       SpaceX CRS-10|             2490|LEO (ISS)|NASA (CRS)|        Success|Success (ground pad)|
+----------+----------+---------------+-----------+--------------------+-----------------+---------+------

In [55]:
spark.stop()

## Author : Sourav Das


## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
